<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-и-изучение" data-toc-modified-id="Загрузка-и-изучение-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка и изучение</a></span></li><li><span><a href="#Предварительная-обработка-данных" data-toc-modified-id="Предварительная-обработка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предварительная обработка данных</a></span><ul class="toc-item"><li><span><a href="#Очистка-данных,-Токенизация,-Лемматизация" data-toc-modified-id="Очистка-данных,-Токенизация,-Лемматизация-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Очистка данных, Токенизация, Лемматизация</a></span><ul class="toc-item"><li><span><a href="#WordNetLemmatizer" data-toc-modified-id="WordNetLemmatizer-1.2.1.1"><span class="toc-item-num">1.2.1.1&nbsp;&nbsp;</span>WordNetLemmatizer</a></span></li></ul></li><li><span><a href="#Векторизация" data-toc-modified-id="Векторизация-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Векторизация</a></span><ul class="toc-item"><li><span><a href="#TF-IDF" data-toc-modified-id="TF-IDF-1.2.2.1"><span class="toc-item-num">1.2.2.1&nbsp;&nbsp;</span>TF-IDF</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#LogisticRegression-+-TF-IDF" data-toc-modified-id="LogisticRegression-+-TF-IDF-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>LogisticRegression + TF-IDF</a></span></li><li><span><a href="#LinearSVC" data-toc-modified-id="LinearSVC-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LinearSVC</a></span></li><li><span><a href="#CatBoost-на-неподготовленных-данных" data-toc-modified-id="CatBoost-на-неподготовленных-данных-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoost на неподготовленных данных</a></span></li><li><span><a href="#LGBM" data-toc-modified-id="LGBM-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>LGBM</a></span></li></ul></li><li><span><a href="#Выбор-модели" data-toc-modified-id="Выбор-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выбор модели</a></span></li><li><span><a href="#Тестирование" data-toc-modified-id="Тестирование-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

## Подготовка

In [1]:
%%capture
!pip install torch
!pip install transformers
!pip install catboost

In [39]:
import numpy as np
import pandas as pd
import re
import os
from time import time
from scipy.sparse import vstack

import spacy
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", 
                    datefmt= '%H:%M:%S', level=logging.INFO)

import torch
import transformers

from catboost import CatBoostClassifier, cv, Pool
from lightgbm import LGBMClassifier

from tqdm import notebook
from tqdm import tqdm

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.model_selection import (cross_val_score, 
                                     train_test_split, 
                                     GridSearchCV,
                                     StratifiedKFold,
                                     RepeatedStratifiedKFold,
                                     RandomizedSearchCV
                                    )
 
from sklearn.metrics import f1_score, make_scorer

from collections import Counter
import multiprocessing
from pprint import pprint

N_JOBS = 4

In [3]:
cores = multiprocessing.cpu_count()
cores

8

### Загрузка и изучение

In [4]:
def data_read(df_name):

    data = pd.read_csv(f'C:\\Users\\m5612\\Downloads\\{df_name}.csv', 
    index_col=[0])

    return data

In [5]:
data = data_read('toxic_comments')
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [6]:
data.describe().T

,count,mean,std,min,25%,50%,75%,max
toxic,159292.0,0.101612,0.302139,0.0,0.0,0.0,0.0,1.0


In [7]:
data.isnull().sum()

text     0
toxic    0
dtype: int64

Пропусков в данных нет, есть дисбаланс по целевому признаку.

### Предварительная обработка данных

В этом этапе данные очищаются от нежелательных символов, приводятся к нижнему регистру и токенизируются (разбиваются на отдельные слова). Кроме того, может проводиться удаление стоп-слов (часто встречающихся слов, которые не несут информации) и лемматизация (приведение слов к их базовой форме).

#### Очистка данных, Токенизация, Лемматизация

##### WordNetLemmatizer

In [8]:
# загружаем стоп-слова
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jovyan/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [9]:
# создаем объект для лемматизации слов
lemmatizer = WordNetLemmatizer()

In [10]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [11]:
# функция для очистки и предварительной обработки текста
def preprocess_text(text):
    # удаляем все символы, кроме букв и цифр
    text = re.sub(r'[^\w\s]', ' ', text)
    # приводим к нижнему регистру
    text = text.lower()
    # токенизируем текст
    words = word_tokenize(text)
    # лемматизируем слова
    words = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in words]
    # удаляем стоп-слова
    words = [word for word in words if not word in stop_words]
    # объединяем слова в строку
    text = ' '.join(words)
    if len(text) > 1:
        return text
    return None

In [12]:
tqdm.pandas()

In [13]:
%%time
data['text'] = data['text'].progress_apply(preprocess_text)

100%|██████████| 159292/159292 [20:31<00:00, 129.30it/s]

CPU times: user 18min 21s, sys: 1min 49s, total: 20min 11s
Wall time: 20min 31s


In [14]:
data = data.dropna()

In [15]:
data['text'].head(5).to_dict()

{0: 'explanation edits make username hardcore metallica fan revert vandalism closure gas vote new york doll fac please remove template talk page since retire 89 205 38 27',
 1: 'aww match background colour seemingly stuck thanks talk 21 51 january 11 2016 utc',
 2: 'hey man really try edit war guy constantly remove relevant information talk edits instead talk page seem care format actual info',
 3: 'make real suggestion improvement wonder section statistic later subsection type accident think reference may need tidy exact format ie date format etc later one else first preference format style reference want please let know appear backlog article review guess may delay reviewer turn list relevant form eg wikipedia good_article_nominations transport',
 4: 'sir hero chance remember page'}

#### Векторизация

Этап векторизации включает преобразование текстовых данных в числовые векторы, которые могут быть использованы для обучения моделей машинного обучения.

Существует несколько способов векторизации текстовых данных, но наиболее распространенными являются:

- Мешок слов (bag-of-words): Это простой метод, который представляет каждый документ (или предложение) как набор слов без учета порядка слов в тексте. Сначала составляется словарь всех уникальных слов в корпусе текстовых данных, а затем каждый документ представляется в виде вектора, где каждый элемент соответствует слову в словаре, а значение элемента указывает на количество вхождений этого слова в документ. Векторы полученные с помощью мешка слов могут быть очень большими, но они могут быть уменьшены с помощью методов сокращения размерности, таких как сингулярное разложение (SVD) или метод главных компонент (PCA).

- TF-IDF (Term Frequency - Inverse Document Frequency): Это более продвинутый метод векторизации, который также использует мешок слов. Он учитывает не только частоту слова в документе, но и его важность в корпусе текстовых данных. Таким образом, слова, которые часто встречаются в документе, но редко в других документах, имеют более высокий вес.

- Word2Vec: Это метод, который использует нейронные сети для преобразования слов в векторы фиксированной длины. Каждое слово представляется в виде вектора, который учитывает контекст, в котором оно находится. Это позволяет сохранять смысловые отношения между словами в векторах. Word2Vec может быть обучен на больших наборах текстовых данных, что позволяет получить более точные представления слов.

- BERT (Bidirectional Encoder Representations from Transformers): Это метод, который использует глубокие нейронные сети, основанные на архитектуре Transformer, для получения представлений текстовых данных. Он учитывает контекст и порядок слов в предложении, что позволяет ему получать более точные представления. BERT был обучен на огромных наборах текстовых данных и может быть использован для решения различных задач, таких как классификация текста, вопросно-ответные системы и машинный перевод.

##### TF-IDF

In [16]:
X_train_valid, X_test, y_train_valid, y_test = train_test_split(data['text'], 
                                                                data['toxic'], 
                                                                test_size=0.1, 
                                                                random_state=42)

In [17]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, 
                                                      y_train_valid, 
                                                      test_size=0.1, 
                                                      random_state=42)

In [18]:
print("Размер X_train_valid:", X_train_valid.shape)
print("Размер y_train_valid:", y_train_valid.shape)
print("Размер X_train:", X_train.shape)
print("Размер X_test:", X_test.shape)
print("Размер y_train:", y_train.shape)
print("Размер y_test:", y_test.shape)

Размер X_train_valid: (143336,)
Размер y_train_valid: (143336,)
Размер X_train: (129002,)
Размер X_test: (15927,)
Размер y_train: (129002,)
Размер y_test: (15927,)


In [19]:
%%time
count_tf_idf = TfidfVectorizer() 
X_train_tfidf = count_tf_idf.fit_transform(X_train) 
X_test_tfidf = count_tf_idf.transform(X_test)
X_valid_tfidf = count_tf_idf.transform(X_valid)
X_train_valid_tfidf = count_tf_idf.transform(X_train_valid)
print("Размер матрицы X_train_tfidf:", X_train_tfidf.shape)
print("Размер матрицы X_test_tfidf:", X_test_tfidf.shape)
print("Размер матрицы X_valid_tfidf:", X_valid_tfidf.shape)
print("Размер матрицы X_train_valid_tfidf:", X_train_valid_tfidf.shape)

Размер матрицы X_train_tfidf: (129002, 151004)
Размер матрицы X_test_tfidf: (15927, 151004)
Размер матрицы X_valid_tfidf: (14334, 151004)
Размер матрицы X_train_valid_tfidf: (143336, 151004)
CPU times: user 9.08 s, sys: 189 ms, total: 9.26 s
Wall time: 9.28 s


In [20]:
X_train_tfidf

<129002x151004 sparse matrix of type '<class 'numpy.float64'>'
	with 3486364 stored elements in Compressed Sparse Row format>

## Обучение

### LogisticRegression + TF-IDF

In [29]:
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("model_lr", LogisticRegression())
    ]
)
pipeline

Pipeline(steps=[('vect', TfidfVectorizer()),
                ('model_lr', LogisticRegression())])

In [51]:
# Задание сетки параметров
parameter_grid = {
    "vect__max_df": (0.4, 0.6, 0.8, 1.0),
    "vect__min_df": (1, 3, 5),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigrams
    "vect__norm": ("l1", "l2"),
    'model_lr__C': [0.1, 1, 10],
    'model_lr__solver': ['saga']
}

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=parameter_grid,
    scoring='f1',
    cv=5,
    n_iter=12,
    random_state=42,
    n_jobs=-1,
    verbose=1,
)

print("Performing grid search...")
print("Hyperparameters to be evaluated:")
pprint(parameter_grid)

Performing grid search...
Hyperparameters to be evaluated:
{'model_lr__C': [0.1, 1, 10],
 'model_lr__solver': ['saga'],
 'vect__max_df': (0.6, 0.8, 1.0),
 'vect__min_df': (1, 3, 5),
 'vect__ngram_range': ((1, 1), (1, 2)),
 'vect__norm': ('l1', 'l2')}


In [52]:
%%time
t0 = time()
random_search.fit(X_train_valid, y_train_valid)
print(f"Done in {time() - t0:.3f}s")

Fitting 5 folds for each of 10 candidates, totalling 50 fits
Done in 834.445s
CPU times: user 13min 31s, sys: 21.8 s, total: 13min 53s
Wall time: 13min 54s


In [53]:
print("Best parameters combination found:")
best_parameters = random_search.best_estimator_.get_params()
for param_name in sorted(parameter_grid.keys()):
    print(f"{param_name}: {best_parameters[param_name]}")

Best parameters combination found:
model_lr__C: 10
model_lr__solver: saga
vect__max_df: 0.6
vect__min_df: 3
vect__ngram_range: (1, 1)
vect__norm: l2


In [58]:
f1_lr = random_search.best_score_
model_lr_best = random_search.best_estimator_
# Выводим наилучшую комбинацию гиперпараметров и соответствующую ей F1-меру на валидационной выборке
print("Best parameters:", random_search.best_params_)
print("Best F1 score:", f1_lr)

Best parameters: {'vect__norm': 'l2', 'vect__ngram_range': (1, 1), 'vect__min_df': 3, 'vect__max_df': 0.6, 'model_lr__solver': 'saga', 'model_lr__C': 10}
Best F1 score: 0.773666816402159


In [59]:
model_lr_best

Pipeline(steps=[('vect', TfidfVectorizer(max_df=0.6, min_df=3)),
                ('model_lr', LogisticRegression(C=10, solver='saga'))])

### LinearSVC

Основное отличие между LinearSVC и LogisticRegression заключается в функции потерь, которую они оптимизируют. LinearSVC использует функцию потерь "hinge", которая хорошо работает на задачах с линейно разделимыми классами, но может давать неопределенные результаты на задачах с пересекающимися классами. В то же время, LogisticRegression использует функцию потерь "logistic", которая хорошо работает на задачах с пересекающимися классами, но может давать неопределенные результаты на задачах с линейно разделимыми классами.

Кроме того, LinearSVC обычно более ресурсоемкий и медленный, чем LogisticRegression, особенно при работе с большими наборами данных. Но при этом он может быть более устойчив к выбросам и шумам в данных, поскольку он оптимизирует границу максимальной ширины (maximum margin).

В отличие от LogisticRegression, LinearSVC не имеет параметра регуляризации L1, но вместо этого имеет параметр регуляризации C, который управляет силой регуляризации. Также LinearSVC может использовать различные функции ядра для работы с нелинейными задачами классификации.

In [60]:
pipeline = Pipeline(
    [
        ("vect", TfidfVectorizer()),
        ("model_svc", LinearSVC(random_state=42))
    ]
)
pipeline

Pipeline(steps=[('vect', TfidfVectorizer()),
                ('model_svc', LinearSVC(random_state=42))])

In [61]:
# Задание сетки параметров
parameter_grid = {
    "vect__max_df": (0.4, 0.6, 0.8, 1.0),
    "vect__min_df": (1, 3, 5),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigrams
    "vect__norm": ("l1", "l2"),
    'model_svc__C': [0.1, 0.5, 1, 5, 10],
}

random_search = RandomizedSearchCV(
    estimator=pipeline,
    param_distributions=parameter_grid,
    scoring='f1',
    cv=5,
    n_iter=12,
    random_state=42,
    n_jobs=-1,
    verbose=1,
)

print("Performing grid search...")
print("Hyperparameters to be evaluated:")
pprint(parameter_grid)

Performing grid search...
Hyperparameters to be evaluated:
{'model_svc__C': [0.1, 0.5, 1, 5, 10],
 'vect__max_df': (0.4, 0.6, 0.8, 1.0),
 'vect__min_df': (1, 3, 5),
 'vect__ngram_range': ((1, 1), (1, 2)),
 'vect__norm': ('l1', 'l2')}


In [62]:
%%time
t0 = time()
random_search.fit(X_train_valid, y_train_valid)
print(f"Done in {time() - t0:.3f}s")

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Done in 868.980s
CPU times: user 14min 10s, sys: 16.7 s, total: 14min 27s
Wall time: 14min 28s


In [63]:
print("Best parameters combination found:")
best_parameters = random_search.best_estimator_.get_params()
for param_name in sorted(parameter_grid.keys()):
    print(f"{param_name}: {best_parameters[param_name]}")

Best parameters combination found:
model_svc__C: 1
vect__max_df: 0.6
vect__min_df: 1
vect__ngram_range: (1, 1)
vect__norm: l2


In [64]:
f1_svc = random_search.best_score_
model_svc_best = random_search.best_estimator_
# Выводим наилучшую комбинацию гиперпараметров и соответствующую ей F1-меру на валидационной выборке
print("Best parameters:", random_search.best_params_)
print("Best F1 score:", f1_svc)

Best parameters: {'vect__norm': 'l2', 'vect__ngram_range': (1, 1), 'vect__min_df': 1, 'vect__max_df': 0.6, 'model_svc__C': 1}
Best F1 score: 0.7770091704993187


### CatBoost на неподготовленных данных

In [66]:
df_train_valid, df_test = train_test_split(data, test_size=0.1, random_state=42)
df_train, df_valid = train_test_split(df_train_valid, test_size=0.1, random_state=42)

In [67]:
model_cb = CatBoostClassifier(verbose=100,
                              eval_metric='F1',
                              random_state=42
                             )

In [68]:
%%time
model_cb.fit(df_train[['text']], df_train[['toxic']],
         eval_set=(df_valid[['text']], df_valid[['toxic']]),
         text_features=['text'])

Learning rate set to 0.105251
0:	learn: 0.7027743	test: 0.7481962	best: 0.7481962 (0)	total: 517ms	remaining: 8m 36s
100:	learn: 0.7435512	test: 0.7551900	best: 0.7562500 (85)	total: 51s	remaining: 7m 33s
200:	learn: 0.7602787	test: 0.7609034	best: 0.7609034 (191)	total: 1m 39s	remaining: 6m 36s
300:	learn: 0.7708763	test: 0.7615714	best: 0.7637213 (260)	total: 2m 30s	remaining: 5m 48s
400:	learn: 0.7797177	test: 0.7617567	best: 0.7637213 (260)	total: 3m 20s	remaining: 4m 59s
500:	learn: 0.7864427	test: 0.7630148	best: 0.7642718 (463)	total: 4m 11s	remaining: 4m 10s
600:	learn: 0.7910320	test: 0.7637490	best: 0.7654130 (558)	total: 5m 4s	remaining: 3m 21s
700:	learn: 0.7955702	test: 0.7645922	best: 0.7654130 (558)	total: 5m 56s	remaining: 2m 32s
800:	learn: 0.7986508	test: 0.7625677	best: 0.7654130 (558)	total: 6m 49s	remaining: 1m 41s
900:	learn: 0.8013135	test: 0.7634533	best: 0.7654130 (558)	total: 7m 41s	remaining: 50.8s
999:	learn: 0.8038012	test: 0.7650696	best: 0.7654130 (558)	t

In [69]:
# предсказание меток классов для тестовой выборки
predictions = model_cb.predict(df_valid[['text']])

In [70]:
# вычисление метрики F1 на тестовой выборке
f1_cb = f1_score(df_valid[['toxic']], predictions)

# вывод результата метрики F1
print('F1 score:', f1_cb)

F1 score: 0.765412950756107


### LGBM

In [71]:
%%time
# Определение параметров модели
params = {
    'random_state': 42
}

# Создание и обучение модели
model = LGBMClassifier(**params)
model.fit(X_train_tfidf, y_train,
          eval_set=[(X_valid_tfidf, y_valid)],
          eval_metric='f1',
         )

[1]	valid_0's binary_logloss: 0.280899
[2]	valid_0's binary_logloss: 0.259441
[3]	valid_0's binary_logloss: 0.24495
[4]	valid_0's binary_logloss: 0.233832
[5]	valid_0's binary_logloss: 0.22502
[6]	valid_0's binary_logloss: 0.217763
[7]	valid_0's binary_logloss: 0.212201
[8]	valid_0's binary_logloss: 0.206687
[9]	valid_0's binary_logloss: 0.202187
[10]	valid_0's binary_logloss: 0.19842
[11]	valid_0's binary_logloss: 0.195053
[12]	valid_0's binary_logloss: 0.191622
[13]	valid_0's binary_logloss: 0.188378
[14]	valid_0's binary_logloss: 0.185514
[15]	valid_0's binary_logloss: 0.182795
[16]	valid_0's binary_logloss: 0.179944
[17]	valid_0's binary_logloss: 0.177864
[18]	valid_0's binary_logloss: 0.175433
[19]	valid_0's binary_logloss: 0.173198
[20]	valid_0's binary_logloss: 0.171044
[21]	valid_0's binary_logloss: 0.16962
[22]	valid_0's binary_logloss: 0.167643
[23]	valid_0's binary_logloss: 0.165948
[24]	valid_0's binary_logloss: 0.164697
[25]	valid_0's binary_logloss: 0.16333
[26]	valid_0's

LGBMClassifier(random_state=42)

In [72]:
# Предсказание вероятностей
y_pred = model.predict(X_valid_tfidf)

# Вычисление значения F1-меры
f1_lgb = f1_score(y_valid, y_pred)

# Вывод значения метрики на валидационной выборке
print('Test F1-score:', f1_lgb)

Test F1-score: 0.7571884984025559


## Выбор модели

In [73]:
pd.DataFrame(columns=['model', 'f1'], 
             data={
                 'model':['LinearRegression', 'LinearSVC', 'LGBM', 'CatBoost'], 
                 'f1':[f1_lr, f1_svc, f1_lgb, f1_cb]})

,model,f1
0,LinearRegression,0.773667
1,LinearSVC,0.777009
2,LGBM,0.757188
3,CatBoost,0.765413


Лучшая модель: LinearSVC с гиперпараметрами по умолчанию

## Тестирование

In [74]:
# Предсказание меток классов на тестовой выборке
y_pred = model_svc_best.predict(X_test)

# Вычисление значения F1-меры на тестовой выборке
f1_svc_test = f1_score(y_test, y_pred)

# Вывод значения метрики на тестовой выборке
print('Test F1-score:', f1_svc_test)

Test F1-score: 0.7674183776506227


Требование по минималльному f1=0.75 выполнено

## Выводы

В рамках проекта была разработана модель классификации комментариев на позитивные и негативные с целью автоматической модерации токсичных комментариев в интернет-магазине. Были выполнены все этапы проекта, включая предобработку данных, обучение и тестирование моделей, выбор оптимальной модели и достижение требуемой метрики F1 не меньше 0.75.

В дальнейшем, для улучшения качества модели и расширения ее функционала, можно рассмотреть следующие направления развития:

1. Улучшение предобработки данных. Возможны дополнительные методы очистки текстов от лишних символов, оптимизация лемматизации, использование более точных алгоритмов векторизации.

2. Оптимизация параметров модели. Можно рассмотреть оптимизацию параметров модели для улучшения ее точности и скорости работы. Например, можно провести более широкий поиск по сетке оптимальных параметров модели и применить нейросети.

3. Расширение функционала модерации. Можно рассмотреть возможность расширения функционала модерации, включая определение других категорий комментариев, например, спам или оскорбительные высказывания.

4. Интеграция модели в рабочее окружение магазина. После успешного тестирования модели необходимо интегрировать ее в рабочее окружение магазина, чтобы модерация комментариев происходила автоматически и эффективно.

Таким образом, дальнейшее развитие проекта может быть направлено на улучшение качества модели, расширение ее функционала и интеграцию в рабочее окружение магазина для обеспечения автоматической модерации комментариев.